In [1]:
!nvidia-smi #using GPU is faster

Thu Sep  5 06:08:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!sudo apt-get install ghostscript
!pip install transformers pythainlp python-crfsuite opencv-python easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9
  libgs9-common libidn12 libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 39 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Do you want to continue? [Y/n] ^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 70.7 MB/s eta 0:00:00:

In [3]:
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
import pandas as pd

tagger = NamedEntityTagger()

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

## MULTIPLE PDFs

In [22]:
import os
import easyocr
import pandas as pd
import shutil
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
from tempfile import TemporaryDirectory

# Define the function to extract text from images
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'])
    result = reader.readtext(image_path)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Convert PDF to images
def convert_pdf_to_images(pdf_path, output_dir):
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    output_pattern = os.path.join(output_dir, f"{pdf_filename}_%d.png")
    os.system(f'gs -dBATCH -dNOPAUSE -sDEVICE=pngalpha -r300 -sOutputFile="{output_pattern}" "{pdf_path}"')

# Function to split text into chunks based on token length
def chunk_text(text, tokenizer, max_tokens=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokenizer.convert_tokens_to_string(tokens[i:i+max_tokens])
        chunks.append(chunk)
    return chunks

# Tag and clean text with chunking
def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern):
    text_chunks = chunk_text(text, tokenizer)
    
    tagged_text = ""
    cleaned_text = ""

    for chunk in text_chunks:
        ner = tagger.get_ner(chunk, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        
        tagged_text += ner + "\n"
        cleaned_text += cleaned_ner + "\n"

    return tagged_text.strip(), cleaned_text.strip()

# Define the function to process and clean multiple PDFs
def process_pdfs_in_directory(pdf_dir):
    tagger = NamedEntityTagger()
    tokenizer = tagger.tokenizer

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    results = []
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, pdf_file)

            with TemporaryDirectory() as temp_dir:
                images_dir = os.path.join(temp_dir, 'images')
                os.makedirs(images_dir, exist_ok=True)

                # Convert the PDF to images
                convert_pdf_to_images(pdf_path, images_dir)

                # Extract text from all images related to the PDF file
                raw_text = ""
                pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
                for img_file in os.listdir(images_dir):
                    if img_file.startswith(pdf_filename):
                        img_path = os.path.join(images_dir, img_file)
                        raw_text += extract_text_from_image(img_path) + "\n"

                # Tag and clean the extracted text using chunking
                tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)

                # Append the result to the list
                results.append({
                    "PDF File": pdf_file,
                    "Raw_Text": raw_text,
                    "Tagged_Text": tagged_text,
                    "Cleaned_Text": cleaned_text
                })

    # Create a DataFrame with all results
    df = pd.DataFrame(results)
    return df


In [23]:
# Example usage
pdf_dir = '/kaggle/input/resume'
df = process_pdfs_in_directory(pdf_dir)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


GPL Ghostscript 10.03.1 (2024-05-02)
Copyright (C) 2024 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


GPL Ghostscript 10.03.1 (2024-05-02)
Copyright (C) 2024 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


...

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


GPL Ghostscript 10.03.1 (2024-05-02)
Copyright (C) 2024 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


GPL Ghostscript 10.03.1 (2024-05-02)
Copyright (C) 2024 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [24]:
df

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการวิเ...
1,AI-engineer-resume-example.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...
2,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\nnattamon. rado...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...
3,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...
4,resume 1.pdf,สมหญิง\n แก้วกาจญ์\nวิศวกรปัญญาประดิษฐ์\nal ที...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปในกา...
5,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...
6,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ ทํางาน วิศวกร a...
7,Resume baa.pdf,ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3 ...,<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...
8,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit.m@dom...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทางาน financial ana...
9,resume 3.pdf,re sume\n ประวัติส่วนตัว\n090-123-4567\npimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ที่มีประสบการณ์ 4 ฺdี ผู้ เชี่ยวชาญด้าน ในด้าน...


In [25]:
print(df['Cleaned_Text'].iloc[9])

ที่มีประสบการณ์ 4 ฺdี ผู้ เชี่ยวชาญด้าน ในด้านการพัฒนาโซลูชันการเรียนรู้ของ วิศวกรปัญญาประดิษฐ์ สําหรับการวิเคราะห์ ข้อมูล เครืองและ ai โดยใช้ scikit learn และ keras มี ธุรกิจ ทักษะ ความสามารถ .. ความสามารถในการจัดการข้อมูลขนาด ใหญ่และสร้างโมเดลที่ช่วยเพิ่ม ภาษา1โปรแกรม python c++ ประสิทธิภาพในการตัดสินใจเชิงธุรกิจ การประมวลผลภาพ opencv pillow การเรียนรู้เชิงลึก tensorflow. keras ประวัติการทํางาน การจัดการข้อมูล sol solite วิศวกร ปัจจุบัน  พัฒนาโมเดล machine learning เพือ ประกาศนียบัตร การพยากรณ์ทางการเงินที่ช่วยให้การ ตัดสินใจเชิงกลยุทธ์มีความแม่นยําขึ้น google cloud professional data engineer 30% ร่วมมือกับทีมวิเคราะห์ ข้อมูลเพือทําการ coursera machine learning by จัดการข้อมูลและสร้างรายงานที่มี stanford ประสิทธิภาพ สําหรับผู้บริหาร ข้อมูลการศึกษา ผู้ ช่วยวิจัย ai ศึกษานารี มัธยมศึกษา   อุดมศึกษา มหาวิทยาลัยธรรมศาสตร์


## SINGLE PDF

In [19]:
import os
import easyocr
import pandas as pd
import shutil
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
from tempfile import TemporaryDirectory

# Define the function to extract text from images
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'])
    result = reader.readtext(image_path)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Convert PDF to images
def convert_pdf_to_images(pdf_path, output_dir):
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    output_pattern = os.path.join(output_dir, f"{pdf_filename}_%d.png")
    os.system(f'gs -dBATCH -dNOPAUSE -sDEVICE=pngalpha -r300 -sOutputFile="{output_pattern}" "{pdf_path}"')

# Function to split text into chunks based on token length
def chunk_text(text, tokenizer, max_tokens=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokenizer.convert_tokens_to_string(tokens[i:i+max_tokens])
        chunks.append(chunk)
    return chunks

# Modified tag and clean text function with chunking
def tag_and_clean_text(text, tagger, tokenizer):
    text_chunks = chunk_text(text, tokenizer)
    
    tagged_text = ""
    cleaned_text = ""

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    for chunk in text_chunks:
        # Tag each chunk using the model
        ner = tagger.get_ner(chunk, tag=True)
        
        # Clean the tagged text
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        
        # Append results from the chunk to the overall text
        tagged_text += ner + "\n"
        cleaned_text += cleaned_ner + "\n"

    return tagged_text.strip(), cleaned_text.strip()

# Define the function to process and clean a single PDF
def process_single_pdf(pdf_path):
    with TemporaryDirectory() as temp_dir:
        # Create a temporary directory for images
        images_dir = os.path.join(temp_dir, 'images')
        os.makedirs(images_dir, exist_ok=True)

        # Convert the PDF to images in the temporary directory
        convert_pdf_to_images(pdf_path, images_dir)

        # Initialize the Named Entity Tagger
        tagger = NamedEntityTagger()
        tokenizer = tagger.tokenizer

        # Extract text from all images related to the PDF file
        raw_text = ""
        pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
        for img_file in os.listdir(images_dir):
            if img_file.startswith(pdf_filename):
                img_path = os.path.join(images_dir, img_file)
                raw_text += extract_text_from_image(img_path) + "\n"

        # Tag and clean the extracted text using the chunking approach
        tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer)

        # Create a DataFrame with the result
        data = {
            "PDF File": os.path.basename(pdf_path),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text
        }
        df = pd.DataFrame([data])

        return df


In [20]:
# Example usage
test = '/kaggle/input/resume/AI-engineer-resume-example.pdf'
test1 = process_single_pdf(test)

GPL Ghostscript 10.03.1 (2024-05-02)
Copyright (C) 2024 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 510). Running this sequence through the model will result in indexing errors
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [21]:
print(test1['Cleaned_Text'].iloc[0])

alexander jones artificial inigence engineer innovative artificial inigence engineer with 12+ years of experience in application design development testing and deployment. highly experienced in writing codes and algorithms as well as building complex neural. networks through various programming languages. possess an unbridled passion for artificial inigence with comprehensive knowledge of machine learning concepts and other related technologies. unmatched abilities to identify. understand and translate program requirements into sustainable javascript python through c# programs for continuous improvement of advanced technical and other solutions c++ ai in work experience areas of expertise artificial inigence engineer ai engineering software prototyping the arts of artificial game tech solutions decision analytics scrum & agile methodologies 05/2017
base through galaxy combat mobile game application (2015) artificial inigence and big data analysis and programming languages expertise alo